In [ ]:
#Library Calls
using Gen
using LinearAlgebra

In [ ]:
#I'll make my own damn HMC sampler, with blackjack and hookers
function jon_hmc()
   return("There's nothing here") 
end

In [ ]:
#There's very little meat in these auxiliary momentum variables

In [ ]:
#See all those gradients, Homer? Thats why your HMC sampler didn't work!
